In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=620d62719fb3099b3ce883499e99c0d9192793e5d8ccb5600f4d88851d116a57
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.1 MB/s eta 0:00:00


In [5]:
pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=14d83a0925771ead8d069cd509306560280d3fddd8aaf04f0b026398b70fd93b
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [11]:
pip install mplfinance

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy
import math
import pandas_ta as ta
from backtesting import Backtest
from backtesting import Strategy


def find_levels(
        price: np.array, atr: float, # Log closing price, and log atr
        first_w: float = 0.1,
        atr_mult: float = 3.0,
        prom_thresh: float = 0.1):


    # Setup weights
    last_w = 1.0
    w_step = (last_w - first_w) / len(price)
    weights = first_w + np.arange(len(price)) * w_step
    weights[weights < 0] = 0.0

    # Get kernel of price.
    kernal = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile
    min_v = np.min(price)
    max_v = np.max(price)
    step = (max_v - min_v) / 200
    price_range = np.arange(min_v, max_v, step)
    pdf = kernal(price_range) # Market profile

    # Find significant peaks in the market profile
    pdf_max = np.max(pdf)
    prom_min = pdf_max * prom_thresh

    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)
    levels = []
    for peak in peaks:
        levels.append(np.exp(price_range[peak]))

    return levels, peaks, props, price_range, pdf, weights
def support_resistance_levels(
        data: pd.DataFrame, lookback: int,
        first_w: float = 0.01, atr_mult:float=3.0, prom_thresh:float =0.25):


    # Get log average true range,
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    all_levels = [None] * len(data)
    for i in range(lookback, len(data)):
        i_start  = i - lookback
        vals = np.log(data.iloc[i_start+1: i+1]['close'].to_numpy())
        levels, peaks, props, price_range, pdf, weights= find_levels(vals, atr.iloc[i], first_w, atr_mult, prom_thresh)
        all_levels[i] = levels

    return all_levels
def sr_penetration_signal(data: pd.DataFrame, levels: np.array):
    signal = np.zeros(len(data))
    curr_sig = 0.0
    close_arr = data['close'].to_numpy()
    for i in range(1, len(data)):
        if levels[i] is None:
            continue

        last_c = close_arr[i - 1]
        curr_c = close_arr[i]


        for level in levels[i]:
            if curr_c > level and last_c <= level: # Close cross above line
                curr_sig = 1.0
            elif curr_c < level and last_c >= level: # Close cross below line
                curr_sig = -1.0

        signal[i] = curr_sig
    return signal
def take_trade(data: pd.DataFrame,signal: np.array):
    long_trades = []
    short_trades = []

    close_arr = data['close'].to_numpy()
    last_sig = 0.0
    open_trade = None
    idx = data.index
    for i in range(len(data)):
        if signal[i] == 1.0 and last_sig != 1.0: # Long entry
            if open_trade is not None:
                open_trade[2] = idx[i]
                open_trade[3] = close_arr[i]
                long_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], 1, np.nan]
        if signal[i] == -1.0  and last_sig != -1.0: # Short entry
            if open_trade is not None:
                open_trade[2] = idx[i]
                open_trade[3] = close_arr[i]
                short_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], 1, np.nan]
        else:
            if data['open'][i]<data['close'][i] and (data['high'][i]-data['close'][i])<(data['open'][i]-data['low'][i]):
                if open_trade is not None:
                  open_trade[2] = idx[i]
                  open_trade[3] = close_arr[i]
                  long_trades.append(open_trade)
            elif data['open'][i]>data['close'][i] and (data['high'][i]-data['open'][i])<(data['close'][i]-data['low'][i]):
                if open_trade is not None:
                  open_trade[2] = idx[i]
                  open_trade[3] = close_arr[i]
                  short_trades.append(open_trade)



        last_sig = signal[i]

    long_trades = pd.DataFrame(long_trades, columns=['entry_datetime', 'entry_price', 'exit_datetime', 'exit_price'])
    short_trades = pd.DataFrame(short_trades, columns=['entry_datetime', 'entry_price', 'exit_datetime', 'exit_price'])
    long_trades = long_trades.set_index('entry_datetime')
    short_trades = short_trades.set_index('entry_datetime')
    long_trades.drop_duplicates(inplace=True)
    short_trades.drop_duplicates(inplace=True)

    return long_trades, short_trades
if __name__ == '__main__':
   url='https://drive.google.com/file/d/1ablFbgH1Tn3OX85dqdzJkYWaX6dtHWx1/view?usp=drive_link'
   file_id=url.split('/')[-2]
   dwn_url='https://drive.google.com/uc?id='+file_id
   data=pd.read_csv(dwn_url)
   data['datetime'] = data['datetime'].astype('datetime64[s]')
   data = data.set_index('datetime')
   data['levels']=support_resistance_levels(data, 365, first_w=1.0, atr_mult=3.0)
   data['pn_signal']=sr_penetration_signal(data, data['levels'].to_numpy())
   long_trades, short_trades =take_trade(data,data['pn_signal'].to_numpy())
   pnl1=0
   pnl2=0
   for i in range(len(long_trades)):
    pnl1=pnl1+(long_trades['exit_price'][i]-long_trades['entry_price'][i])
   for i in range(len(short_trades)):
    pnl2=pnl2+(short_trades['entry_price'][i]-short_trades['exit_price'][i])
   print('LONG TRADES')
   print( long_trades.head(10))
   print('SHORT TRADES')
   print( short_trades.head(10))




LONG TRADES
                     entry_price       exit_datetime  exit_price
entry_datetime                                                  
2018-01-19 07:30:00     10575.14 2018-01-19 09:30:00    10979.77
2018-01-19 09:30:00     10979.77 2018-01-19 22:30:00    11235.93
2018-01-19 22:30:00     11235.93 2018-01-20 01:30:00    11284.63
2018-01-20 01:30:00     11284.63 2018-01-20 02:30:00    11230.97
2018-01-20 02:30:00     11230.97 2018-01-20 04:30:00    11474.98
2018-01-20 04:30:00     11474.98 2018-01-21 20:30:00    11412.00
2018-01-21 20:30:00     11412.00 2018-01-22 00:30:00    11356.99
2018-01-22 01:30:00     11029.96 2018-01-22 04:30:00    11530.00
2018-01-22 04:30:00     11530.00 2018-01-22 16:30:00    11240.00
2018-01-22 16:30:00     11240.00 2018-01-24 17:30:00    11316.38
SHORT TRADES
                     entry_price       exit_datetime  exit_price
entry_datetime                                                  
2018-01-19 09:30:00     10979.77 2018-01-19 22:30:00    11235.93
